In [1]:
import random
from deap import base, creator, tools, algorithms
import numpy as np

# Problem constants
NDIM = 2
BOUND_LOW, BOUND_UP = 0.0, 5.0

# Creator for fitness (Minimize both objectives)
creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, BOUND_LOW, BOUND_UP)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=NDIM)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    x1, x2 = individual
    f1 = x1 ** 2
    f2 = (x2 - 2) ** 2
    # Constraint handling
    g = x1 + x2 - 1  # Constraint g(x) >= 0
    penalty = 0 if g >= 0 else -100 * g  # Penalize if g < 0
    return f1 + penalty, f2 + penalty

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0)
toolbox.register("mutate", tools.mutPolynomialBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0, indpb=1.0/NDIM)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    pop = toolbox.population(n=100)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    algorithms.eaMuPlusLambda(pop, toolbox, mu=100, lambda_=200, cxpb=0.6, mutpb=0.3, ngen=40,
                              stats=stats, halloffame=hof, verbose=True)

    return pop, stats, hof

if __name__ == "__main__":
    main()


gen	nevals	avg                  	std                    	min                    	max                      
0  	100   	[8.4964024 2.2083985]	[7.11888845 2.62228002]	[0.00050611 0.00012223]	[23.89257996 20.3182078 ]
1  	180   	[1.84218357 2.12962233]	[2.64341955 2.01204521]	[0.00031572 0.00012223]	[15.17809517  8.6273927 ]
2  	174   	[0.43379372 1.5912753 ]	[1.11240646 1.73541371]	[1.59121662e-05 9.61840932e-05]	[10.18721378  8.6273927 ]
3  	180   	[0.07525395 2.08014221]	[0.16060306 2.39686504]	[1.59121662e-05 1.22234284e-04]	[1.05573176 8.6273927 ]  
4  	179   	[0.00923808 2.07169568]	[0.03213948 2.58910419]	[1.59121662e-05 9.17652580e-05]	[0.24838667 8.6273927 ]  
5  	178   	[8.95659986e-04 2.58410120e+00]	[0.00632521 2.95103535]	[7.33262229e-06 9.17652580e-05]	[0.06378727 7.36319514]  
6  	177   	[1.46215983e-04 2.06248715e+00]	[1.40387955e-04 2.63377579e+00]	[1.83538445e-06 9.17652580e-05]	[9.80577887e-04 8.02420590e+00]
7  	182   	[4.36892034e-05 1.41064326e+00]	[5.69946131e-05 2.3